In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import yfinance as yf
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

%config InlineBackend.figure_format = 'svg'

In [ ]:
torch.manual_seed(0)
device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [ ]:
# df = yf.download("EEM", end="2025-01-01", interval="1d",multi_level_index=False)
# df.to_csv("../data/raw/eem.csv", index=True)

In [ ]:
df = pd.read_csv("../data/raw/eem.csv", parse_dates=True)

In [ ]:
df["Close"].plot(title="EEM", figsize=(12, 8))

In [ ]:
class GRU(nn.Module):
    def __init__(self, input_size, hidden_layer_size, num_layers):
        super(GRU, self).__init__()
        self.num_layers = num_layers
        self.hidden_layer_size = hidden_layer_size
        self.gru = nn.GRU(
            input_size, hidden_layer_size, num_layers=num_layers, batch_first=True
        )
        self.fc = nn.Linear(hidden_layer_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_layer_size).to(device)
        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
def create_sequences(data, window_size=7):
    xs, ys = [], []
    for i in range(len(data) - window_size):
        x = data[i : (i + window_size)]
        y = data[i + window_size]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

In [ ]:
num_inputs = 10
gru = GRU(1, hidden_layer_size=64, num_layers=4).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(gru.parameters(), lr=0.005)
epochs = 1000

In [ ]:
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

In [ ]:
Xy_train_val, Xy_test = train_test_split(
    df["Close"].values, test_size=0.2, shuffle=False
)
Xy_train, Xy_val = train_test_split(Xy_train_val, test_size=0.2, shuffle=False)

In [ ]:
X_train, y_train = create_sequences(Xy_train, window_size=num_inputs)
X_val, y_val = create_sequences(Xy_val, window_size=num_inputs)
X_test, y_test = create_sequences(Xy_test, window_size=num_inputs)

X_train = x_scaler.fit_transform(X_train)
X_val = x_scaler.transform(X_val)
X_test = x_scaler.transform(X_test)

y_train = y_scaler.fit_transform(y_train.reshape(-1, 1)).reshape(-1)
y_val = y_scaler.transform(y_val.reshape(-1, 1)).reshape(-1)
y_test = y_scaler.transform(y_test.reshape(-1, 1)).reshape(-1)

In [ ]:
def train_model(hyperparams, X_train, y_train, X_val, y_val):
    hidden_size = hyperparams["hidden_size"]
    num_layers = hyperparams["num_layers"]
    learning_rate = hyperparams["lr"]
    epochs = hyperparams["epochs"]

    model = GRU(1, hidden_layer_size=hidden_size, num_layers=num_layers).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss = nn.MSELoss()

    X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(-1).to(device)
    y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(-1).to(device)
    X_val = torch.tensor(X_val, dtype=torch.float32).unsqueeze(-1).to(device)
    y_val = torch.tensor(y_val, dtype=torch.float32).unsqueeze(-1).to(device)

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        y_pred = model(X_train)
        train_loss = loss(y_pred, y_train)
        train_loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            model.eval()
            with torch.no_grad():
                y_val_pred = model(X_val)
                val_loss = loss(y_val_pred, y_val)
            print(
                f"Epoch {epoch}, Train Loss: {train_loss.item()}, Val Loss: {val_loss.item()}"
            )
    return model

In [ ]:
# Hyperparameter tuning
def objective(trial, seed=0):
    hidden_size = trial.suggest_int("hidden_size", 16, 128)
    num_layers = trial.suggest_int("num_layers", 1, 4)
    learning_rate = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    epochs = trial.suggest_int("epochs", 100, 1000)
    num_inputs = trial.suggest_int("num_inputs", 5, 30)

    model = GRU(1, hidden_layer_size=hidden_size, num_layers=num_layers).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    X_train, y_train = create_sequences(Xy_train, window_size=num_inputs)
    X_val, y_val = create_sequences(Xy_val, window_size=num_inputs)

    X_train = x_scaler.fit_transform(X_train)
    X_val = x_scaler.transform(X_val)

    y_train = y_scaler.fit_transform(y_train.reshape(-1, 1)).reshape(-1)
    y_val = y_scaler.transform(y_val.reshape(-1, 1)).reshape(-1)

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(-1).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(-1).to(device)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32).unsqueeze(-1).to(device)
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32).unsqueeze(-1).to(device)

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        train_output = model(X_train_tensor)
        train_loss = criterion(train_output, y_train_tensor)

        train_loss.backward()
        optimizer.step()

        if (epoch + 1) % 100 == 0:
            model.eval()
            with torch.no_grad():
                val_output = model(X_val_tensor)
                val_loss = criterion(val_output, y_val_tensor)
            trial.report(val_loss.item(), epoch)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    # Compute final validation loss after training completes
    model.eval()
    with torch.no_grad():
        final_val_output = model(X_val_tensor)
        final_val_loss = criterion(final_val_output, y_val_tensor)

    return final_val_loss.item()


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

In [ ]:
params = study.best_params

In [ ]:
# X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(-1).to(device)
# y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(-1).to(device)
# X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(-1).to(device)
# y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(-1).to(device)

# for epoch in range(epochs):
#     gru.train()
#     optimizer.zero_grad()

#     output = gru(X_train_tensor)
#     train_loss = criterion(output, y_train_tensor)

#     train_loss.backward()
#     optimizer.step()

#     if (epoch + 1) % 100 == 0:
#         gru.eval()
#         with torch.no_grad():
#             test_output = gru(X_test_tensor)
#             test_loss = criterion(test_output, y_test_tensor)
#         print(
#             f"Epoch [{epoch + 1}/{epochs}], Train Loss: {train_loss.item()}, Test Loss: {test_loss.item()}"
#         )

In [ ]:
gru = train_model(params, X_train, y_train, X_val, y_val)

In [ ]:
points = []
for i in range(len(X_test)):
    input_seq = (
        torch.tensor(X_test[i, :], dtype=torch.float32)
        .to(device)
        .unsqueeze(-1)
        .unsqueeze(0)
    )
    value = gru(input_seq)
    points.append(value.cpu().item())
points = y_scaler.inverse_transform(np.array(points).reshape(-1, 1))
print(mean_squared_error(points, y_scaler.inverse_transform(y_test.reshape(-1, 1))))
plt.plot(y_scaler.inverse_transform(y_test.reshape(-1, 1)), label="Observed")
plt.plot(points, "--", label="Predicted")
plt.legend()

In [ ]:
# torch.save(gru, "../models/gru1.pt")